In [1]:
import os
import time
import math
import shutil
import statistics
import numpy as np
import pandas as pd
import yfinance as yf
import pandas_ta as TA
import tensorflow as tf
import keras_preprocessing
import matplotlib.pyplot as plt

from PIL import Image
from keras_preprocessing import image
from datetime import timedelta, datetime
from tensorflow.keras import backend as K
from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import load_model
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, load_img

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
pd.set_option('display.max_columns', None)

# Configuracoes

In [4]:
# GLOBAL PARAMETERS
WINDOW = 11
LENGTH_LIST = range(6,21)

# Global Parameters CNN
IMG_SIZE = 15
EPOCHS = 100
BATCH_SIZE = 128
STEPS_EPOCH = 1260//BATCH_SIZE
PATH_RAIZ = 'C:\\Users\\Gush\\Documents\\USP\\Inteligencia Computacional\\Trabalho\\stocks'

# Real Parameters
START_YEAR = 2010
LAST_YEAR = 2021
YEAR_WINDOW = 5

# Financial Evaluation Parameters
FIN_EV_START = START_YEAR + YEAR_WINDOW
FIN_EV_END = LAST_YEAR
INITIAL_CAPITAL = 10000
INITIAL_STOCKS = 0
COST_TRADING = 1

In [5]:
# lista_acoes = [x.replace('.csv','') for x in os.listdir(r'C:\Users\Gush\Documents\USP\Inteligencia Computacional\Trabalho\csv')]
lista_acoes_dow = ['AAPL', 'AXP', 'BA', 'CAT', 'CSCO', 'CVX', 'DIS', 'GE', 'GS', 'HD', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO',
 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PFE', 'PG', 'TRV', 'UNH', 'VZ', 'WMT', 'XOM']
lista_acoes_b3 = ['VALE3','PETR4','ITUB4','BBDC4','PETR3','BBAS3','ABEV3','ELET3','WEGE3','RENT3']
lista_acoes_b3 = [acao+'.SA' for acao in lista_acoes_b3]
#data = yf.download("DVN", start='2021-09-01', end='2021-09-11')

lista_acoes = lista_acoes_b3

# Tratamento de valores

## Normalizacao por Fechamento

In [6]:
def normalizacao_por_fechamento(df):
    df['Date'] = pd.to_datetime(df['Date'])
    normalize_columns = ['Open','High','Low','Close']
    for column in normalize_columns:
        df[f'Adjusted{column}'] = df[f'{column}'] / df['Adj Close']
    return df

## Labeling

In [7]:
def label(row):
    if row['Adj Close'] == row['High Window']:
        return "SELL"
    elif row['Adj Close'] == row['Low Window']:
        return "BUY"
    else:
        return "HOLD"

def labeling_data(df, window):
    index = 0
    high_window = []
    low_window = []
    num_columns = df.shape[0]
    while num_columns > index:
        maxIndex = max(list(df['Adj Close'].iloc[index:index+window]))
        minIndex = min(list(df['Adj Close'].iloc[index:index+window]))
        high_window.extend([maxIndex for x in range(window)])
        low_window.extend([minIndex for x in range(window)])
        index+=window

    high_window = high_window[:num_columns]
    low_window = low_window[:num_columns]
    df["High Window"] = pd.Series(high_window).values 
    df["Low Window"] = pd.Series(low_window).values

    df['Result'] = df.apply(label, axis=1)
    return df

## Indicadores Técnicos

In [8]:
from collections import deque

class PSAR:

    def __init__(self, length, init_af=0.02, max_af=0.2, af_step=0.02):
        self.max_af = max_af
        self.init_af = init_af
        self.af = init_af
        self.af_step = af_step
        self.extreme_point = None
        self.high_price_trend = []
        self.low_price_trend = []
        self.high_price_window = deque(maxlen=length)
        self.low_price_window = deque(maxlen=length)

        # Lists to track results
        self.psar_list = []
        self.af_list = []
        self.ep_list = []
        self.high_list = []
        self.low_list = []
        self.trend_list = []
        self._num_days = 0

    def calcPSAR(self, high, low):
        if self._num_days >= 3:
            psar = self._calcPSAR()
        else:
            psar = self._initPSARVals(high, low)

        psar = self._updateCurrentVals(psar, high, low)
        self._num_days += 1

        return psar

    def _initPSARVals(self, high, low):
        if len(self.low_price_window) <= 1:
            self.trend = None
            self.extreme_point = high
            return None

        if self.high_price_window[0] < self.high_price_window[1]:
            self.trend = 1
            psar = min(self.low_price_window)
            self.extreme_point = max(self.high_price_window)
        else: 
            self.trend = 0
            psar = max(self.high_price_window)
            self.extreme_point = min(self.low_price_window)

        return psar

    def _calcPSAR(self):
        prev_psar = self.psar_list[-1]
        if self.trend == 1: # Up
            psar = prev_psar + self.af * (self.extreme_point - prev_psar)
            psar = min(psar, min(self.low_price_window))
        else:
            psar = prev_psar - self.af * (prev_psar - self.extreme_point)
            psar = max(psar, max(self.high_price_window))

        return psar

    def _updateCurrentVals(self, psar, high, low):
        if self.trend == 1:
            self.high_price_trend.append(high)
        elif self.trend == 0:
            self.low_price_trend.append(low)

        psar = self._trendReversal(psar, high, low)

        self.psar_list.append(psar)
        self.af_list.append(self.af)
        self.ep_list.append(self.extreme_point)
        self.high_list.append(high)
        self.low_list.append(low)
        self.high_price_window.append(high)
        self.low_price_window.append(low)
        self.trend_list.append(self.trend)

        return psar

    def _trendReversal(self, psar, high, low):
        # Checks for reversals
        reversal = False
        if self.trend == 1 and psar > low:
            self.trend = 0
            psar = max(self.high_price_trend)
            self.extreme_point = low
            reversal = True
        elif self.trend == 0 and psar < high:
            self.trend = 1
            psar = min(self.low_price_trend)
            self.extreme_point = high
            reversal = True

        if reversal:
            self.af = self.init_af
            self.high_price_trend.clear()
            self.low_price_trend.clear()
        else:
                if high > self.extreme_point and self.trend == 1:
                    self.af = min(self.af + self.af_step, self.max_af)
                    self.extreme_point = high
                elif low < self.extreme_point and self.trend == 0:
                    self.af = min(self.af + self.af_step, self.max_af)
                    self.extreme_point = low

        return psar

In [9]:
class Technical_Indicators():
    def __init__(self, df, length_list):
        self.df = df
        self.high = df['High']
        self.low = df['Low']
        self.close = df['Adj Close']
        self.volume = df['Volume']
        self.length_list = length_list

    def calculate_rsi(self, close, length):
        fn_roll = lambda s: s.rolling(length).mean()
        # Get the difference in price from previous step
        delta = close.diff()
        # Get rid of the first row, which is NaN since it did not have a previous row to calculate the differences
        delta = delta[1:] 

        # Make the positive gains (up) and negative gains (down) Series
        up, down = delta.clip(lower=0), delta.clip(upper=0).abs()

        roll_up, roll_down = fn_roll(up), fn_roll(down)
        rs = roll_up / roll_down
        rsi = 100.0 - (100.0 / (1.0 + rs))

        # Avoid division-by-zero if 'roll_down' is zero
        # This prevents inf and/or nan values.
        rsi[:] = np.select([roll_down == 0, roll_up == 0, True], [100, 0, rsi])
        rsi.name = 'rsi'

        # Assert range
        valid_rsi = rsi[length - 1:]
        assert ((0 <= valid_rsi) & (valid_rsi <= 100)).all()
        # Note: rsi[:length - 1] is excluded from above assertion because it is NaN for SMA.

        return rsi

    def calculate_wr(self, high, low, close, length):
        highh = high.rolling(length).max() 
        lowl = low.rolling(length).min()
        wr = -100 * ((highh - close) / (highh - lowl))
        return wr

    def calculate_sma(self, close, length):
        sma_series = close.rolling(window=length).mean()
        return sma_series

    def calculate_ema(self, close, length):
        ema_series = close.ewm(span=length, adjust=False).mean()
        return ema_series

    def calculate_wma(self, close, length):
        wma_series = close.rolling(length).apply(lambda x: ((np.arange(length)+1)*x).sum()/(np.arange(length)+1).sum(), raw=True)
        return wma_series

    def calculate_hma(self, close, length):
        hma_series = self.calculate_wma(self.calculate_wma(close, length//2).multiply(2).sub(self.calculate_wma(close, length)), int(np.sqrt(length)))
        return hma_series

    def calculate_tema(self, close, length):
        tema_series = (3*self.calculate_ema(close,length) - 3*self.calculate_ema(self.calculate_ema(close,length),length)) + self.calculate_ema(self.calculate_ema(self.calculate_ema(close,length),length),length)
        return tema_series

    def calculate_cci(self, high, low, close, length):
        # typical price
        tp = (high + low + close) / 3
        # simple moving average
        sma = tp.rolling(length).mean()
        # mean average deviation
        mad = tp.rolling(length).apply(lambda x: pd.Series(x).mad())
        cci_series = (tp - sma) / (0.015 * mad) 
        return cci_series

    def calculate_cmo(self, close, length):
        cmo_series = TA.cmo(close, length=length, talib=False)
        return cmo_series

    def calculate_macd(self, close, length):
        macd_series = self.calculate_ema(close,12) - self.calculate_ema(close,26)
        return macd_series

    def calculate_ppo(self, close, length):
        ppo_series = 100*(self.calculate_ema(close,12) - self.calculate_ema(close,26)) / self.calculate_ema(close,26)
        return ppo_series

    def calculate_roc(self, close, length):
        latest_close = close
        previous_close = close.shift(length)
        roc_series = 100*(latest_close - previous_close)/(previous_close)
        return roc_series

    def _calculate_money_flow_volume_series(self, high, low, close, volume):
        """
        Calculates money flow series
        """
        mfv = volume * (2*close - high - low) / (high - low)
        return mfv

    def _calculate_money_flow_volume(self, high, low, close, volume, length):
        """
        Calculates money flow volume, or q_t in our formula
        """
        return self._calculate_money_flow_volume_series(high,low,close,volume).rolling(length).sum()

    def _calculate_chaikin_money_flow(self, high, low, close, volume, length):
        """
        Calculates the Chaikin money flow
        """
        return self._calculate_money_flow_volume(high,low,close,volume,length) / volume.rolling(length).sum()

    def calculate_cmfi(self, high, low, close, volume, length):
        cmfi_series = self._calculate_chaikin_money_flow(high,low,close,volume,length)
        return cmfi_series
    
    def calculate_dmi(self, high, low, close, period):
        """
        Computes the ADX indicator.
        """
        data = pd.DataFrame({'High': high, 'Low': low, 'Close': close})

        df = data.copy()
        alpha = 1/period

        # TR
        df['H-L'] = df['High'] - df['Low']
        df['H-C'] = np.abs(df['High'] - df['Close'].shift(1))
        df['L-C'] = np.abs(df['Low'] - df['Close'].shift(1))
        df['TR'] = df[['H-L', 'H-C', 'L-C']].max(axis=1)
        del df['H-L'], df['H-C'], df['L-C']

        # ATR
        df['ATR'] = df['TR'].ewm(alpha=alpha, adjust=False).mean()

        # +-DX
        df['H-pH'] = df['High'] - df['High'].shift(1)
        df['pL-L'] = df['Low'].shift(1) - df['Low']
        df['+DX'] = np.where(
            (df['H-pH'] > df['pL-L']) & (df['H-pH']>0),
            df['H-pH'],
            0.0
        )
        df['-DX'] = np.where(
            (df['H-pH'] < df['pL-L']) & (df['pL-L']>0),
            df['pL-L'],
            0.0
        )
        del df['H-pH'], df['pL-L']

        # +- DMI
        df['S+DM'] = df['+DX'].ewm(alpha=alpha, adjust=False).mean()
        df['S-DM'] = df['-DX'].ewm(alpha=alpha, adjust=False).mean()
        df['+DMI'] = (df['S+DM']/df['ATR'])*100
        df['-DMI'] = (df['S-DM']/df['ATR'])*100
        del df['S+DM'], df['S-DM']

        # ADX
        df['DX'] = (np.abs(df['+DMI'] - df['-DMI'])/(df['+DMI'] + df['-DMI']))*100
        df['ADX'] = df['DX'].ewm(alpha=alpha, adjust=False).mean()
        del df['DX'], df['ATR'], df['TR'], df['-DX'], df['+DX'], df['+DMI'], df['-DMI']
        
        adx_series = df['ADX']
        return adx_series
    
    def calculate_sar(self, high, low, length):
    
        data = pd.DataFrame({'High': high, 'Low': low})

        indic = PSAR(length=length)
        data['PSAR'] = data.apply(lambda x: indic.calcPSAR(x['High'], x['Low']), axis=1)
        sar_series = data['PSAR']
        return sar_series

    def calculate_indicators(self):
        for length in self.length_list:
            self.length = length
            self.df[f'TEC_IND_RSA_{length}_DAYS'] = self.calculate_rsi(self.close, length)
            self.df[f'TEC_IND_WR_{length}_DAYS'] = self.calculate_wr(self.high, self.low, self.close, length)
            self.df[f'TEC_IND_SMA_{length}_DAYS'] = self.calculate_sma(self.close, length)
            self.df[f'TEC_IND_EMA_{length}_DAYS'] = self.calculate_ema(self.close, length)
            self.df[f'TEC_IND_WMA_{length}_DAYS'] = self.calculate_wma(self.close, length)
            self.df[f'TEC_IND_HMA_{length}_DAYS'] = self.calculate_hma(self.close, length)
            self.df[f'TEC_IND_TEMA_{length}_DAYS'] = self.calculate_tema(self.close, length)
            self.df[f'TEC_IND_CCI_{length}_DAYS'] = self.calculate_cci(self.high, self.low, self.close, length)
            self.df[f'TEC_IND_CMO_{length}_DAYS'] = self.calculate_cmo(self.close, length)
            self.df[f'TEC_IND_MACD_{length}_DAYS'] = self.calculate_macd(self.close, length)
            self.df[f'TEC_IND_PPO_{length}_DAYS'] = self.calculate_ppo(self.close, length)
            self.df[f'TEC_IND_ROC_{length}_DAYS'] = self.calculate_roc(self.close, length)
            self.df[f'TEC_IND_CMFI_{length}_DAYS'] = self.calculate_cmfi(self.high, self.low, self.close, self.volume, length)
            self.df[f'TEC_IND_DMI_{length}_DAYS'] = self.calculate_dmi(self.high, self.low, self.close, length)
            self.df[f'TEC_IND_SAR_{length}_DAYS'] = self.calculate_sar(self.high, self.low, length)
        
        return self.df

# Utils

In [10]:
def gera_checkpoint(df,nome):
    df.to_csv(f'{nome}.csv',sep=';',decimal=',')

In [11]:
def delete_stock(path_raiz, acao):
    path_delecao = f'{path_raiz}\\{acao}'
    shutil.rmtree(path_delecao)

## Normalizacao dos indicadores e labeling

In [12]:
def normalize_columns(df):
    colunas_indicadores = [col for col in df.columns if 'TEC_IND_' in col]
    for column in colunas_indicadores:
        df[column] =  (df[column]-df[column].min())/(df[column].max()-df[column].min())  
    return df

# Geração imagens

In [13]:
def image_data(df, length_list):
    all_dates = df['Date'].tolist()
    indicators = ['RSA','WR','SMA','EMA','WMA','HMA','TEMA','CCI','CMO','MACD','PPO','ROC','CMFI','DMI','SAR']
    length_list = length_list
    complete_list = []

    for data_individual in all_dates:
        individual_date_dict = {}
        individual_analyse = df[df['Date'] == data_individual]

        # f'TEC_IND_{nome}_{length}_DAYS'
        macro_list = []
        for indicator in indicators:
            micro_list = []
            for length in length_list:
                micro_list.append(individual_analyse[f'TEC_IND_{indicator}_{length}_DAYS'].iloc[0])
            macro_list.append(micro_list)
        y = np.array([np.array(xi) for xi in macro_list])
        z = (y * 255).astype(np.uint8)

        individual_date_dict['Date'] = data_individual
        individual_date_dict['Image'] = z
        individual_date_dict['Label'] = individual_analyse['Result'].tolist()[0]
        complete_list.append(individual_date_dict)

    df_images = pd.DataFrame(complete_list)
    return df_images

In [14]:
def image_generator(df, acao):
    all_dates = df['Date'].tolist()
    for data in all_dates:
        sub_df_data = df[df['Date']==data]
        byt = sub_df_data['Image'].tolist()[0]
        label = sub_df_data['Label'].tolist()[0]
        ano = sub_df_data['Date'].tolist()[0].strftime('%Y')
        full_date = sub_df_data['Date'].tolist()[0].strftime('%Y%m%d')
        img = Image.fromarray(byt, 'L')
        save_path = f'stocks/{acao}/{ano}/{label}'
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        save_complete_path = save_path + f'/{full_date}.png'
        #print(f'Salvando imagem {save_path_complete}')
        img.save(save_complete_path)
    print(f'As imagens da acao {acao} foram criadas com sucesso')

# CNN

## Funções para manipulação de arquivos

In [15]:
def create_dir(path_raiz, acao, start_year, last_year, tipo):
    operacoes = ['BUY','SELL','HOLD']
    range_years = range(start_year,last_year+1)
    for year in range_years:
        for operacao in operacoes:
            src = f'{path_raiz}\\{acao}\\{year}\\{operacao}\\'
            des = f'{path_raiz}\\{acao}\\{tipo}\\{operacao}\\'
            if not os.path.exists(des):
                os.makedirs(des)
            # print(f'Movendo do diretorio {src} para {dest}')
            src_files = os.listdir(src)
            for file_name in src_files:
                full_file_name = os.path.join(src, file_name)
                if os.path.isfile(full_file_name):
                    shutil.copy(full_file_name, des)

In [16]:
def create_dirs(path_raiz, acao, start_year_train, last_year_train, start_year_test=None, last_year_test=None):
    # Criando treino
    # print(f"Criando path de treino para periodo {start_year_train}-{last_year_train}")
    create_dir(path_raiz, acao, start_year_train, last_year_train, 'train')
    
    if start_year_test is not None:
        if last_year_test is None: 
            last_year_test = start_year_test

        print(f"Criando path de teste para periodo {start_year_test}-{last_year_test}")
        create_dir(path_raiz, acao, start_year_test, last_year_test, 'test')
    

In [17]:
def delete_dirs(path_raiz, acao):
    dirs = ['train','test']

    for diretorio in dirs:
        path_delecao = f'{path_raiz}\\{acao}\\{diretorio}'
        shutil.rmtree(path_delecao)	
        # print(f'Path {path_delecao} deletado')

## Criacao da Rede

### Metricas

In [18]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

### Modelo

In [19]:
def modelo(img_size):
    model = tf.keras.models.Sequential([
        # Primeira camada de convolução
        tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(img_size, img_size, 1)),
        # Segunda camada de convolução
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        # Max Pooling 2x2
        tf.keras.layers.MaxPooling2D(2,2),
        # Primeiro Dropout
        tf.keras.layers.Dropout(0.25),  
        # Camada Flatten / Fully Connected
        tf.keras.layers.Flatten(),
        # Segundo Dropout
        tf.keras.layers.Dropout(0.5),
        # Camada Dense
        tf.keras.layers.Dense(3, activation='softmax')
    ])
    # model.summary()
    return model

### Data Generator

In [20]:
def build_datagenerator(path_raiz,acao,batch_size, img_size, train_or_test):

    generatorImage = ImageDataGenerator(
        rescale=1. / 255
    )

    TEST_DIR = f'{path_raiz}\\{acao}\\{train_or_test}\\'
    generator = generatorImage.flow_from_directory(
        TEST_DIR,
        target_size=(img_size,img_size),
        color_mode='grayscale',
        class_mode='categorical',
        batch_size=batch_size
    )
    label_map = (generator.class_indices)
    # print(label_map)
    return generator

### CNN

In [21]:
def CNN(path_raiz,acao,epocas,steps_per_epoch,batch_size):
    img_size = IMG_SIZE
    class_weight = {0: 9, 1: 1., 2: 9.}
    model = modelo(img_size)
    model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['acc', f1_m, precision_m, recall_m])

    train_generator = build_datagenerator(path_raiz=path_raiz, acao=acao, batch_size=batch_size, img_size=img_size, train_or_test='train')
    test_generator = build_datagenerator(path_raiz=path_raiz, acao=acao, batch_size=batch_size, img_size=img_size, train_or_test='test')
    history = model.fit(train_generator, epochs=epocas, steps_per_epoch=steps_per_epoch, validation_data=test_generator, verbose=0, validation_steps=steps_per_epoch//5, class_weight=class_weight)
    
    return model, history

# Rede Final

## Financial Evaluation

In [22]:
def predicao(model,path):
    img = image.load_img(path, target_size=(IMG_SIZE, IMG_SIZE),color_mode="grayscale")
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    images = np.vstack([x])
    # {'BUY': 0, 'HOLD': 1, 'SELL': 2}
    classes = model.predict(images, verbose=0)
    # print(classes)
    if classes[0][0] > classes[0][1] and classes[0][0] > classes[0][2]:
        classe = 0
        tipo = 'BUY'
    elif classes[0][2] > classes[0][0] and classes[0][2] > classes[0][1]:
        classe = 2
        tipo = 'SELL'
    else:
        classe = 1
        tipo = 'HOLD'
    #print(path)
    #print(tipo)
    return classe, tipo

In [23]:
def create_dir_evaluation(path_raiz,acao,start_year,last_year,tipo):
    operacoes = ['BUY','SELL','HOLD']
    range_years = range(start_year,last_year+1)
    for year in range_years:
        for operacao in operacoes:
            src = f'{path_raiz}\\{acao}\\{year}\\{operacao}\\'
            des = f'{path_raiz}\\{acao}\\{tipo}\\'
            if not os.path.exists(des):
                os.makedirs(des)
            # print(f'Movendo do diretorio {src} para {dest}')
            src_files = os.listdir(src)
            for file_name in src_files:
                full_file_name = os.path.join(src, file_name)
                if os.path.isfile(full_file_name):
                    shutil.copy(full_file_name, des)

In [37]:
def Financial_Evaluation_Manager(df, path_raiz, acao, model, start_year, last_year, initial_capital, initial_stocks, cost_trading, last_day, preco_compra=None):
    tipo = 'FinEv'
    create_dir_evaluation(path_raiz=path_raiz,acao=acao,start_year=start_year,last_year=last_year,tipo=tipo)
    df_trade = df[(df['Date'] >= f'{start_year}-01-01') & (df['Date'] <= f'{last_year}-12-31')] #.drop(['Result'],axis=1)
    df_trade['StockPath'] = f"{path_raiz}\\{acao}\\{tipo}\\" + df['Date'].dt.strftime('%Y%m%d') +".png"
    
    status = ''
    list_stocks = {}
    confusion_matrix = []
    money = initial_capital
    stocks = initial_stocks
    
    print(f"Iniciando ano de trading com {money} dólares e {stocks} ações")
    for index, row in df_trade.iterrows():
        path_image = row['StockPath']
        price_stock = row['Adj Close']
        day = row['Date']
        window_prediction = row['Result']
        predicao_stock, tipo_operacao = predicao(model,path_image)
        
        confusion_matrix.append({'window': window_prediction, 'previsao':tipo_operacao})
        
        if predicao_stock == 0 and money > price_stock and status != 'COM' and stocks = 0:
            stocks = stocks + int(money/price_stock)
            money = money - stocks*price_stock - cost_trading
            status = 'COM'
            preco_compra = price_stock
            print(f'{day} - Operacao de Compra (deveria ser {window_prediction}) - Att: Dinheiro {money} | Acoes {stocks}')
        elif predicao_stock == 2 and stocks > 0 and status != 'SEM' and price_stock > preco_compra:
            money = money + stocks*price_stock - cost_trading
            stocks = 0
            status = 'SEM'
            preco_compra = 0
            print(f'{day} - Operacao de Venda (deveria ser {window_prediction}) - Att: Dinheiro {money} | Acoes {stocks}')
        
        if last_day == day and stocks > 0:
            print('Ultimo dia de negociações - vendendo ações remanecentes')
            money = money + stocks*price_stock - cost_trading
            stocks = 0
            status = 'SEM'
            print(f'{day} - Operacao de Venda (deveria ser {window_prediction})- Att: Dinheiro {money} | Acoes {stocks}')
        
        list_stocks[day] = {'stocks':stocks,'money':money}

    return list_stocks, confusion_matrix, money, stocks, preco_compra
    

## Manager

In [38]:
def CNN_Manager(df, path_raiz, acao, start_year, last_year, year_window,
                epochs, steps_epoch, batch_size, 
                initial_capital, initial_stocks, cost_trading, last_day):
    
    # Inicializa variaveis
    lista_historicos = []
    list_stocks = {}
    list_matrix = []
    
    # Valores iniciais
    start_year_train = start_year
    last_year_train = start_year + year_window - 1
    _capital = initial_capital
    _stocks = initial_stocks
    _preco_compra = None
    
    while last_year_train < last_year:
        print(f"""Periodo de treino {start_year_train}-{last_year_train}\nPeríodo de teste {last_year_train+1}""")
        
        create_dirs(path_raiz,acao,start_year_train,last_year_train,last_year_train+1)

        model, historico = CNN(path_raiz,acao,epochs,steps_epoch,batch_size)
        lista_historicos.append(historico)
        delete_dirs(path_raiz,acao)
        # model.save(f'models//{acao}_ts_to_cnn.h5')
        
        stocks, matrix, _capital, _stocks, _preco_compra = Financial_Evaluation_Manager(df=df, path_raiz=path_raiz, acao=acao, model=model,
                                                                       start_year=last_year_train+1, last_year=last_year_train+1, 
                                                                       initial_capital=_capital, initial_stocks=_stocks, 
                                                                       cost_trading=cost_trading, last_day=last_day, preco_compra=_preco_compra)
        
        list_stocks = {**list_stocks,**stocks}
        list_matrix = [*list_matrix, *matrix]
        
        time.sleep(3)
        
        start_year_train += 1
        last_year_train += 1
        
    return lista_historicos, list_stocks, list_matrix

# BRUTE

In [44]:
lista_acoes=lista_acoes
final_report = {}
for acao in lista_acoes:
    print(f"Iniciando analise para a ação {acao} as {datetime.now().strftime('%d/%m/%y %H:%M:%S')}")
    try:
        delete_stock(path_raiz=PATH_RAIZ, acao=acao)
    except:
        ...
    df = pd.read_csv(f'csv//{acao}.csv')
    df = normalizacao_por_fechamento(df=df)
    dfl = labeling_data(df=df, window=WINDOW)
    dfi = Technical_Indicators(df=dfl, length_list=LENGTH_LIST).calculate_indicators()
    df_indicadores = dfi.copy()
    dfn = normalize_columns(df=dfi)
    df_images = image_data(df=dfn, length_list=LENGTH_LIST)
    image_generator(df=df_images, acao=acao)

    last_day = dfn.iloc[-1]['Date']

    lista_historico, list_stocks, list_matrix = CNN_Manager(df=dfn, path_raiz=PATH_RAIZ, acao=acao, 
                                                           start_year=START_YEAR, last_year=LAST_YEAR, year_window=YEAR_WINDOW, 
                                                           epochs=EPOCHS, steps_epoch=STEPS_EPOCH, batch_size=BATCH_SIZE, 
                                                           initial_capital=INITIAL_CAPITAL, initial_stocks=INITIAL_STOCKS, cost_trading=COST_TRADING,
                                                           last_day=last_day)

    final_report[acao] = {
        'df': df_indicadores,
        'historico': lista_historico,
        'stocks': list_stocks,
        'matrix': list_matrix
    }
    # delete_stock(path_raiz=PATH_RAIZ, acao=acao)
    print(f"Finalizando analise para a ação {acao} as {datetime.now().strftime('%d/%m/%y %H:%M:%S')}")
    

Iniciando analise para a ação VALE3.SA as 21/11/22 19:46:03
As imagens da acao VALE3.SA foram criadas com sucesso
Periodo de treino 2010-2014
Período de teste 2015
Criando path de teste para periodo 2015-2015
Found 1237 images belonging to 3 classes.
Found 246 images belonging to 3 classes.
Iniciando ano de trading com 10000 dólares e 0 ações
2015-03-11 00:00:00 - Operacao de Compra (deveria ser BUY)- Att: Dinheiro 3.2202809999998863 | Acoes 839
2015-03-17 00:00:00 - Operacao de Venda (deveria ser HOLD)- Att: Dinheiro 10538.598743 | Acoes 0
2015-05-19 00:00:00 - Operacao de Compra (deveria ser HOLD)- Att: Dinheiro 6.292934999999488 | Acoes 802
2015-05-26 00:00:00 - Operacao de Venda (deveria ser HOLD)- Att: Dinheiro 10887.294897 | Acoes 0
2015-08-24 00:00:00 - Operacao de Compra (deveria ser BUY)- Att: Dinheiro 1.2817460000005667 | Acoes 1083
2015-08-27 00:00:00 - Operacao de Venda (deveria ser HOLD)- Att: Dinheiro 12503.913124 | Acoes 0
2015-09-28 00:00:00 - Operacao de Compra (deveri

# Avaliacoes Manuais

In [45]:
def create_confusion_matrix(matrix):
    y_actu = []
    y_pred = []
    for x in matrix:
        y_actu.append(x['window'])
        y_pred.append(x['previsao'])
        
    y_actu = pd.Series(y_actu, name='Actual')
    y_pred = pd.Series(y_pred, name='Predicted')
    
    df_confusion = pd.crosstab(y_actu, y_pred)
    return df_confusion

In [46]:
matrix = []
for acao, chave in final_report.items():
    matrix = [*matrix, *chave['matrix']]
    
confusion_matrix = create_confusion_matrix(matrix)
confusion_matrix

Predicted,BUY,HOLD,SELL
Actual,,,
BUY,400,511,747
HOLD,1040,2658,10404
SELL,23,100,1497


In [47]:
for acao, chave in final_report.items():
    print(acao)
    df_index = pd.DataFrame(chave['stocks']).T.reset_index().rename(columns = {'index':'data'})
    dinheiro_no_ano = list(df_index['money'])
    primeiro_dia = 10000
    print(primeiro_dia)
    ultimo_dia = dinheiro_no_ano[-1]
    print(ultimo_dia)
    # i = (FV/PV)^(1/n) - 1
    i = (ultimo_dia/primeiro_dia)**(1/(FIN_EV_END-FIN_EV_START+1)) - 1
    print(f'O percentual de lucro entre {FIN_EV_START} e {FIN_EV_END} foi de {round(i*100,2)}% a.a.')

VALE3.SA
10000
18736.981968000004
O percentual de lucro entre 2015 e 2021 foi de 9.38% a.a.
PETR4.SA
10000
23629.783638
O percentual de lucro entre 2015 e 2021 foi de 13.07% a.a.
ITUB4.SA
10000
184.27608899999598
O percentual de lucro entre 2015 e 2021 foi de -43.48% a.a.
BBDC4.SA
10000
34440.007225000016
O percentual de lucro entre 2015 e 2021 foi de 19.32% a.a.
PETR3.SA
10000
14668.378432000003
O percentual de lucro entre 2015 e 2021 foi de 5.63% a.a.
BBAS3.SA
10000
21477.244162000003
O percentual de lucro entre 2015 e 2021 foi de 11.54% a.a.
ABEV3.SA
10000
8057.849624999997
O percentual de lucro entre 2015 e 2021 foi de -3.04% a.a.
ELET3.SA
10000
21377.249443000004
O percentual de lucro entre 2015 e 2021 foi de 11.46% a.a.
WEGE3.SA
10000
19594.498474000025
O percentual de lucro entre 2015 e 2021 foi de 10.09% a.a.
RENT3.SA
10000
571.7618459999989
O percentual de lucro entre 2015 e 2021 foi de -33.56% a.a.


In [ ]:
import matplotlib.dates as md

y_stocks = list(final_report['ITUB4.SA']['df']['TEC_IND_CMFI_20_DAYS'])
y_money = list(final_report['ITUB4.SA']['df']['Close'])
x = list([x.date() for x in final_report['ITUB4.SA']['df']['Date']])

t = x
data1 = y_stocks 
data2 = y_money

fig, ax1 = plt.subplots()

color = 'tab:gray'
ax1.set_xlabel('Data')
ax1.set_ylabel('RSA', color=color)
#ax1.ylim([50,100])
ax1.plot(t, data1, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:green'
ax2.set_ylabel('Preço ação', color=color)  # we already handled the x-label with ax1
#ax2.ylim([9000,10000])
ax2.plot(t, data2, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

In [ ]:
for acao, chave in final_report.items():
    print(acao)
    df_index = chave['df']
    inicio = df_index[df_index['Date'] >= '01-01-2015'].iloc[0,:]['AdjustedClose']
    # print(inicio)
    fim = df_index.iloc[-1,:]['AdjustedClose']
    # print(fim)
    i = 100*((fim/inicio)**(1/7) - 1)
    print(i)


In [52]:
final_report['BBAS3.SA']['historico'][0].history

{'loss': [2.7941386699676514,
  2.7078540325164795,
  2.58170485496521,
  2.5345211029052734,
  2.43554949760437,
  2.381488084793091,
  2.4313433170318604,
  2.4194064140319824,
  2.3583807945251465,
  2.380258321762085,
  2.2983829975128174,
  2.254361391067505,
  2.2609899044036865,
  2.3748345375061035,
  2.286350965499878,
  2.354246139526367,
  2.274785280227661,
  2.2756526470184326,
  2.3620924949645996,
  2.2598447799682617,
  2.2591121196746826,
  2.2730095386505127,
  2.2870876789093018,
  2.2377617359161377,
  2.273521661758423,
  2.2197301387786865,
  2.227388620376587,
  2.2768993377685547,
  2.279895305633545,
  2.276456594467163,
  2.2518951892852783,
  2.2606165409088135,
  2.195847511291504,
  2.2278125286102295,
  2.21513295173645,
  2.2652177810668945,
  2.251331329345703,
  2.245591402053833,
  2.161745548248291,
  2.136592388153076,
  2.1650755405426025,
  2.0995516777038574,
  2.237673759460449,
  2.1340885162353516,
  2.1868536472320557,
  2.2085249423980713,
  